In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/Users/felix/Github/DataSci/ml/KaggleML/melb_data.csv')

In [23]:
# Selecting X and y
y = data.Price
X = data.drop(['Price'], axis=1)

# Dividing into training and validation
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Selecting categoricals with low cardinality. Arbitrary but convenient
categorical_cols = [col for col in X_train_full.columns if 
                    X_train_full[col].nunique() < 10
                   and X_train_full[col].dtype == 'object']

# Selecting numerical columns
numerical_cols = [col for col in X_train_full.columns if 
                  X_train_full[col].dtype in ['int64', 'float64']]

# Keep / concatenate selected columns only 
the_cols = categorical_cols + numerical_cols
X_train = X_train_full[the_cols].copy()
X_valid = X_valid_full[the_cols].copy()



In [27]:
# Defining preprocessing steps

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data. Imputing null values
numerical_transformer = SimpleImputer(strategy = 'constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [28]:
# Define the model. Here we used a RandomForestRegressor class
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)


In [32]:
# Create and evaluate the pipeline

# This bundles the preprocessing and modelling steps
# With the pipeline, training data is preprocessed and the model is fitted in one line
# Without the pipeline, imputation, OHE, and training are done separately 
# We pass the unprocessed X_valid features to predict(), and the pipeline automatically preprocesses before predicting

from sklearn.metrics import mean_absolute_error

# Define pipe steps as bundled preprocessing and model instantiation
da_pipe = Pipeline(steps =[('preprocessor', preprocessor), 
                           ('model', model)
                          ])

# Fit and predict the pipeline
da_pipe.fit(X_train, y_train)
preds = da_pipe.predict(X_valid)

# Evaluation
score = mean_absolute_error(y_valid, preds)

In [34]:
print(f"MAE: \t {score}")

MAE: 	 160679.18917034855
